In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
X_train = torch.load('../input/nfltrainimagecrop/train_ds_no_mask_crops.pt')
y_train = torch.load('../input/nfltrainimagecrop/train_ds_no_mask_labels.pt').type(torch.LongTensor)
X_test = torch.load('../input/nfltrainimagecrop/test_ds_no_mask_crops.pt')
y_test = torch.load('../input/nfltrainimagecrop/test_ds_no_mask_labels.pt').type(torch.LongTensor)

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, X, y, transform=None):
        'Initialization'
        self.X = X
        self.y = y
        self.transform = transform

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.y)

  def __getitem__(self, index):
        'Generates one sample of data'
        X = self.X[index]
        y = self.y[index]
        X = self.transform(X)
        return X, y

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = Dataset(X_train, y_train,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = Dataset(X_test, y_test,transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('no-impact','impact')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[int(labels[j])] for j in range(4)))

In [ ]:
images.shape

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 32)
        self.fc4 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if (i % 100 == 99) and (epoch % 5 == 4):    # print every 2000 mini-batches
        if (i % 100 == 99) :    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './classifier_net.pth'


In [ ]:
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[int(labels[j])] for j in range(4)))

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
outputs = net(images)


In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
y_pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred += predicted
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
print(net)

In [ ]:
from  sklearn.metrics import classification_report
print(classification_report(y_test,np.array(y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_test.numpy(), np.array(y_pred))

cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Reload the saved dataset.
df_submission = pd.read_csv('../input/nlfimpactdfsubmission/template_submission.csv')
df_submission['video'] = df_submission['video'] + '.mp4'
del df_submission['Unnamed: 0']
df_submission.reset_index(inplace = True)
df_submission

In [ ]:
def predictor(X, index):
    X = transform(X)
    outputs = net(X)
    _, predicted = torch.max(outputs.data, 1)
    return pd.Series(predicted, index=index)

In [ ]:
_data = torch.load('../input/crop-for-submission/submission_imgs_no_mask_crops_0.pt')
df_submission.loc[:10000-1,'impact'] = predictor(_data, index=df_submission.loc[:10000-1,'index'])

_data = torch.load('../input/crop-for-submission/submission_imgs_no_mask_crops_1.pt')
df_submission.loc[10000:20000-1,'impact'] = predictor(_data, index=df_submission.loc[10000:20000-1,'index'])

_data = torch.load('../input/crop-for-submission/submission_imgs_no_mask_crops_2.pt')
df_submission.loc[20000:30000-1,'impact'] = predictor(_data, index=df_submission.loc[20000:30000-1,'index'])

_data = torch.load('../input/crop-for-submission/submission_imgs_no_mask_crops_3.pt')
df_submission.loc[30000:40000-1,'impact'] = predictor(_data, index=df_submission.loc[30000:40000-1,'index'])

_data = torch.load('../input/crop-for-submission/submission_imgs_no_mask_crops_4.pt')
df_submission.loc[40000:50000-1,'impact'] = predictor(_data, index=df_submission.loc[40000:50000-1,'index'])

_data = torch.load('../input/crop-for-submission/submission_imgs_no_mask_crops_5.pt')
df_submission.loc[50000:,'impact'] =  predictor(_data, index=df_submission.loc[50000:,'index'])

In [ ]:
df_submission.to_csv()
df_submission

In [ ]:
np.sum(df_submission['impact'])

In [ ]:
df_submission_impact = df_submission.loc[df_submission['impact']==1,['gameKey','playID','view','video','frame','left','width','top','height']]
df_submission_impact['left'] =df_submission_impact['left'].astype(int)
df_submission_impact['width'] =df_submission_impact['width'].astype(int)
df_submission_impact['top'] =df_submission_impact['top'].astype(int)
df_submission_impact['height'] =df_submission_impact['height'].astype(int)
df_submission_impact.reset_index(inplace=True, drop=True)

df_submission_impact.to_csv()
df_submission_impact

In [ ]:
import nflimpact
env = nflimpact.make_env()
env.predict(df_submission_impact) # df is a pandas dataframe of your entire submission file

In [ ]:
imshow(_data[0])

In [ ]:
pd.read_csv('./submission.csv').head()